In [1]:
# === Generic State Gerrymandering Analysis Script Template ===
#
# Instructions for use:
# 1. Copy this entire file and rename it for the state you are analyzing
#    (e.g., analyze_newyork.py, analyze_texas.py).
# 2. VERY CAREFULLY review and update ALL variables in the
#    "--- STATE-SPECIFIC CONFIGURATION - UPDATE FOR EACH STATE ---" section.
#    This is the MOST CRITICAL step. Errors here will cause the script to fail
#    or produce incorrect results.
# 3. Ensure all data files listed in the paths exist and are accessible.
# 4. Run the script from your terminal: python analyze_yourstate.py
#
# Outputs will be saved to a 'results/STATE_ABBR/' directory.
#
# Required libraries: pandas, geopandas, numpy, matplotlib
# Ensure they are installed: pip install pandas geopandas numpy matplotlib

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import logging

# --- Configure Logging ---
# Clears previous handlers to avoid duplicate messages if re-running in a notebook
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# --- STATE-SPECIFIC CONFIGURATION - UPDATE FOR EACH STATE ---
# V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V
#                          MUST BE UPDATED FOR EACH STATE
# V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V V

STATE_ABBR = "TX"  # e.g., "NY", "TX" - Used for creating output folder names
STATE_NAME = "Texas"  # e.g., "New York", "Texas"

# --- File Paths ---
# (Adjust these paths relative to where you run the script, or use absolute paths)
# It's recommended to have a base data directory and then state-specific subdirectories.
# Example: project_root/data/NY/voter_files/NY_voter_file.csv
BASE_DATA_PATH = f"../StateData/TexasData/" # Adjusted for running from StateAnalysis folder
VOTER_FILE_PATH = os.path.join(BASE_DATA_PATH, "TX_2022_prim_l2_vf_2020blocks/TX_2022_prim_l2_vf_2020blocks.csv") # VERIFY THIS FILENAME
CVAP_FILE_PATH = os.path.join(BASE_DATA_PATH, "tx_cvap_2022_2020_b/tx_cvap_2022_2020_b.csv") # VERIFY THIS FILENAME
DISTRICT_SHAPEFILE_PATH = os.path.join(BASE_DATA_PATH, "tx_cong_2021/PLANC2193.shp") # VERIFY THIS FILENAME (e.g., Plan C2185 or similar)
TRACT_SHAPEFILE_PATH = os.path.join(BASE_DATA_PATH, "tx_t_2020_bound/tx_t_2020_bound.shp") # VERIFY THIS FILENAME

# --- Target CRS for Geometric Calculations (Area, Perimeter) ---
# Find a suitable projected CRS for the state (e.g., State Plane or UTM zone)
# Search for "[State Name] projected CRS EPSG"
TARGET_CRS_EPSG = "EPSG:XXXXX" # e.g., "EPSG:32118" for NY Long Island (State Plane)

# --- Key Column Names (VERIFY THESE AGAINST YOUR ACTUAL FILES) ---
# Voter File Columns
VF_BLOCK_ID_COL = 'geoid20'         # 15-digit block GEOID in voter file (or equivalent)
VF_TOTAL_REG_COL = 'total_reg'
VF_PARTY_DEM_COL = 'party_dem'
VF_PARTY_REP_COL = 'party_rep'
VF_PARTY_NPP_COL = 'party_npp'      # Non-Partisan/Other - use if available, otherwise script can handle its absence
VF_GENERAL_2020_VOTES_COL = 'g20201103' # L2 column for 2020 General Election turnout (or relevant election)
# Add any specific election result columns if needed for renaming for your state
# Example: 'p20220524': 'Primary_2022'
VF_ELECTION_RENAME_DICT = {
    'g20201103': 'General_2020_Turnout' # Example, ensure VF_GENERAL_2020_VOTES_COL matches a key here if renaming
    # Add other election renames specific to this state's L2 file if you use them
}


# CVAP File Columns
CVAP_BLOCK_ID_COL = 'GEOID20'       # 15-digit block GEOID in CVAP file
CVAP_TOTAL_COL = 'CVAP_TOT22'
CVAP_WHITE_COL = 'CVAP_WHT22'
CVAP_BLACK_COL = 'CVAP_BLK22'
CVAP_HISPANIC_COL = 'CVAP_HSP22'
CVAP_ASIAN_COL = 'CVAP_ASN22'
CVAP_AIAN_COL = 'CVAP_AIA22'        # American Indian/Alaska Native
CVAP_NHPI_COL = 'CVAP_NHP22'        # Native Hawaiian/Pacific Islander
# Add other CVAP groups if needed, e.g., CVAP_2OM22 (Two or More Races)

# Tract Shapefile Columns
TRACT_SHP_ID_COL = 'GEOID20'        # Typically the 11-digit tract GEOID in tract shapefile

# District Shapefile Columns
DISTRICT_SHP_ID_COL = 'District'    # Or 'CD118FP', 'DISTRICTNO', 'GEOID', etc. VERIFY!

# --- Community Concentration Analysis Parameters ---
COMMUNITY_THRESHOLD_PCT = 30.0
# Define which minority groups to analyze and their corresponding CVAP percentage column names
# (these 'pct_...' columns will be created during CVAP processing)
MINORITY_GROUPS_FOR_ANALYSIS = {
    # Internal key : Name of the 'pct_group' column that will be created
    'black': 'pct_black',
    'hispanic': 'pct_hispanic',
    'asian': 'pct_asian',
    # 'aian': 'pct_aian', # Uncomment and ensure CVAP_AIAN_COL is set if you want to analyze
    # 'nhpi': 'pct_nhpi'  # Uncomment and ensure CVAP_NHPI_COL is set
}
# Map these internal keys to the raw CVAP count column names from your CVAP file
# This helps in aggregating raw counts to districts and calculating overall district %
CVAP_COLUMN_MAP_FOR_GROUPS = {
    'black': CVAP_BLACK_COL,
    'hispanic': CVAP_HISPANIC_COL,
    'asian': CVAP_ASIAN_COL,
    'aian': CVAP_AIAN_COL,
    'nhpi': CVAP_NHPI_COL
    # Add other groups if defined in MINORITY_GROUPS_FOR_ANALYSIS
}

# ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^
#                          END OF STATE-SPECIFIC CONFIGURATION
# ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^

# --- Output Directory ---
RESULTS_DIR = f"../results/{STATE_ABBR}/"
os.makedirs(RESULTS_DIR, exist_ok=True)

# --- Helper Functions (Should not need modification for different states) ---

def calculate_polsby_popper(gdf_districts_input, district_id_col, target_crs):
    """Calculates Polsby-Popper scores."""
    logging.info("Calculating Polsby-Popper scores...")
    gdf_districts_scores = gdf_districts_input.copy()
    try:
        gdf_districts_proj = gdf_districts_scores.to_crs(target_crs)
    except Exception as e:
        logging.warning(f"Error reprojecting for Polsby-Popper: {e}. Using original CRS (may be less accurate).")
        gdf_districts_proj = gdf_districts_scores
        
    gdf_districts_proj['area'] = gdf_districts_proj.geometry.area
    gdf_districts_proj['perimeter'] = gdf_districts_proj.geometry.length
    gdf_districts_proj['polsby_popper'] = 0.0
    valid_perimeter = gdf_districts_proj['perimeter'] > 0
    # Avoid division by zero or issues with very small perimeters
    gdf_districts_proj.loc[valid_perimeter, 'polsby_popper'] = \
        (4 * np.pi * gdf_districts_proj.loc[valid_perimeter, 'area']) / \
        (gdf_districts_proj.loc[valid_perimeter, 'perimeter']**2).replace(0, np.nan) # Avoid division by zero
    gdf_districts_scores['polsby_popper'] = gdf_districts_proj['polsby_popper'].fillna(0)
    return gdf_districts_scores[[district_id_col, 'polsby_popper']]

def calculate_efficiency_gap(gdf_districts_input, district_id_col, dem_votes_col, rep_votes_col):
    """Calculates the Efficiency Gap."""
    logging.info("Calculating Efficiency Gap...")
    if dem_votes_col not in gdf_districts_input.columns or rep_votes_col not in gdf_districts_input.columns:
        logging.error(f"Vote columns ('{dem_votes_col}', '{rep_votes_col}') not found in district data for EG calc.")
        return np.nan, pd.DataFrame(columns=['DISTRICT', 'Dem_Votes', 'Rep_Votes', 'Total_Votes', 'Wasted_Dem_Votes', 'Wasted_Rep_Votes'])

    df_results = gdf_districts_input[[district_id_col, dem_votes_col, rep_votes_col]].copy()
    df_results.rename(columns={district_id_col: 'DISTRICT', dem_votes_col: 'Dem_Votes', rep_votes_col: 'Rep_Votes'}, inplace=True)

    df_results['Total_Votes'] = df_results['Dem_Votes'] + df_results['Rep_Votes']
    df_results['Win_Threshold'] = np.floor(df_results['Total_Votes'] / 2) + 1
    df_results['Wasted_Dem_Votes'] = 0.0
    df_results['Wasted_Rep_Votes'] = 0.0

    dem_wins = df_results['Dem_Votes'] > df_results['Rep_Votes']
    rep_wins = df_results['Rep_Votes'] > df_results['Dem_Votes']
    tie = df_results['Dem_Votes'] == df_results['Rep_Votes']

    df_results.loc[dem_wins, 'Wasted_Dem_Votes'] = df_results['Dem_Votes'] - df_results['Win_Threshold']
    df_results.loc[rep_wins | tie, 'Wasted_Dem_Votes'] = df_results['Dem_Votes']
    df_results.loc[rep_wins, 'Wasted_Rep_Votes'] = df_results['Rep_Votes'] - df_results['Win_Threshold']
    df_results.loc[dem_wins | tie, 'Wasted_Rep_Votes'] = df_results['Rep_Votes']

    df_results['Wasted_Dem_Votes'] = df_results['Wasted_Dem_Votes'].clip(lower=0)
    df_results['Wasted_Rep_Votes'] = df_results['Wasted_Rep_Votes'].clip(lower=0)

    total_votes_statewide = df_results['Total_Votes'].sum()
    total_wasted_dem = df_results['Wasted_Dem_Votes'].sum()
    total_wasted_rep = df_results['Wasted_Rep_Votes'].sum()

    efficiency_gap_value = (total_wasted_rep - total_wasted_dem) / total_votes_statewide if total_votes_statewide > 0 else 0
    return efficiency_gap_value, df_results

def calculate_community_concentration(gdf_tracts_with_district_input, district_id_col, minority_pct_cols_map, threshold_pct):
    """Calculates community concentration."""
    logging.info(f"Calculating Community Concentration (Threshold: >{threshold_pct}%) ...")
    gdf_tracts_with_district = gdf_tracts_with_district_input.copy()
    concentration_results = []
    distribution_details_list = []
    
    if district_id_col not in gdf_tracts_with_district.columns:
        logging.error(f"District ID column '{district_id_col}' not found in input GeoDataFrame for comm. conc.")
        return pd.DataFrame(), pd.DataFrame() # Return empty DataFrames
        
    num_districts = gdf_tracts_with_district[district_id_col].nunique()
    if num_districts == 0:
        logging.warning("No districts found. Skipping community concentration.")
        return pd.DataFrame(), pd.DataFrame()
    baseline_even_spread = 1 / num_districts

    all_districts = sorted(gdf_tracts_with_district[district_id_col].unique())
    district_summary_base = pd.DataFrame({district_id_col: all_districts})

    for group_key, pct_col_name in minority_pct_cols_map.items():
        group_name_display = group_key.capitalize()
        high_conc_col = f'high_{group_key}_cvap_{threshold_pct}'

        if pct_col_name not in gdf_tracts_with_district.columns:
            logging.warning(f"CVAP percentage column '{pct_col_name}' for group '{group_name_display}' not found. Skipping this group.")
            concentration_results.append({
                'Minority Group': group_name_display, 'Threshold (%)': threshold_pct,
                'Total High-Conc Tracts Statewide': 0, 'Concentration Score (Sum Sq Shares)': np.nan,
                'Baseline Even Spread': baseline_even_spread, 'Interpretation': 'Skipped - Data missing'
            })
            empty_df = district_summary_base.copy()
            empty_df[f'high_{group_key}_tracts'] = 0
            empty_df[f'share_statewide_high_{group_key}'] = 0.0
            distribution_details_list.append(empty_df[[district_id_col, f'high_{group_key}_tracts', f'share_statewide_high_{group_key}']])
            continue

        gdf_tracts_with_district.loc[:, high_conc_col] = gdf_tracts_with_district[pct_col_name] > threshold_pct
        total_high_conc_tracts_statewide = gdf_tracts_with_district[high_conc_col].sum()
        logging.info(f"Analyzing: {group_name_display} (using {pct_col_name}) - Found {total_high_conc_tracts_statewide} tracts statewide >{threshold_pct}% CVAP.")

        high_tracts_col_name = f'high_{group_key}_tracts'
        share_col_name = f'share_statewide_high_{group_key}'

        if total_high_conc_tracts_statewide > 0:
            dist_group_summary = gdf_tracts_with_district[gdf_tracts_with_district[high_conc_col]].groupby(district_id_col).size().reset_index(name=high_tracts_col_name)
            dist_group_summary = pd.merge(district_summary_base, dist_group_summary, on=district_id_col, how='left').fillna(0)
            dist_group_summary[high_tracts_col_name] = dist_group_summary[high_tracts_col_name].astype(int)
            dist_group_summary[share_col_name] = dist_group_summary[high_tracts_col_name] / total_high_conc_tracts_statewide
            
            concentration_score = (dist_group_summary[share_col_name]**2).sum()
            
            mid_point = (baseline_even_spread + 1.0) / 2
            lower_bound = baseline_even_spread + (mid_point - baseline_even_spread) * 0.33 # Adjusted thresholds slightly
            upper_bound = mid_point + (1.0 - mid_point) * 0.33 # Adjusted thresholds slightly

            if concentration_score >= upper_bound : interpretation = f"High Concentration / Packing (Score: {concentration_score:.3f})"
            elif concentration_score <= lower_bound: interpretation = f"Dispersed / Potential Cracking (Score: {concentration_score:.3f})"
            else: interpretation = f"Moderate Concentration / Uneven Split (Score: {concentration_score:.3f})"
        else:
            concentration_score = np.nan
            interpretation = "No tracts above threshold"
            dist_group_summary = district_summary_base.copy()
            dist_group_summary[high_tracts_col_name] = 0
            dist_group_summary[share_col_name] = 0.0
            
        distribution_details_list.append(dist_group_summary[[district_id_col, high_tracts_col_name, share_col_name]])
        concentration_results.append({
            'Minority Group': group_name_display, 'Threshold (%)': threshold_pct,
            'Total High-Conc Tracts Statewide': total_high_conc_tracts_statewide,
            'Concentration Score (Sum Sq Shares)': concentration_score,
            'Baseline Even Spread': baseline_even_spread, 'Interpretation': interpretation
        })
        gdf_tracts_with_district.drop(columns=[high_conc_col], inplace=True, errors='ignore')

    final_distribution_df = district_summary_base.copy()
    for detail_df in distribution_details_list:
        # Ensure the detail_df has the district_id_col before merging
        if district_id_col in detail_df.columns:
            final_distribution_df = pd.merge(final_distribution_df, detail_df, on=district_id_col, how='left')
        else:
            logging.warning(f"Skipping merge for a detail_df as it's missing district_id_col: {district_id_col}")

    return pd.DataFrame(concentration_results), final_distribution_df.fillna(0)

# --- Main Analysis Logic ---
def main():
    global DISTRICT_SHP_ID_COL
    logging.info(f"Starting analysis for {STATE_NAME} ({STATE_ABBR})")

    # 1. Load Voter File Data
    logging.info(f"Loading voter file: {VOTER_FILE_PATH}")
    try:
        df_vf = pd.read_csv(VOTER_FILE_PATH, low_memory=False) # low_memory=False can help with mixed types
        # Apply renames if dict is provided and not empty
        if VF_ELECTION_RENAME_DICT:
            df_vf.rename(columns=VF_ELECTION_RENAME_DICT, inplace=True)
            # Update VF_GENERAL_2020_VOTES_COL if it was renamed
            global VF_GENERAL_2020_VOTES_COL 
            VF_GENERAL_2020_VOTES_COL = VF_ELECTION_RENAME_DICT.get(VF_GENERAL_2020_VOTES_COL, VF_GENERAL_2020_VOTES_COL)

    except FileNotFoundError:
        logging.error(f"Voter file not found at {VOTER_FILE_PATH}. Please check the path in the configuration.")
        return # Exit if critical file is missing

    df_active = df_vf[df_vf[VF_TOTAL_REG_COL] > 0].copy() # Use .copy() to avoid SettingWithCopyWarning
    df_active.loc[:, VF_BLOCK_ID_COL] = df_active[VF_BLOCK_ID_COL].astype(str).str.zfill(15)
    df_active.loc[:, 'census_tract'] = df_active[VF_BLOCK_ID_COL].str[:11]

    # Calculate estimated votes
    # Handle cases where party columns might be missing (e.g., some states might not have NPP)
    df_active.loc[:, 'dem_proportion'] = (df_active[VF_PARTY_DEM_COL] / df_active[VF_TOTAL_REG_COL]).fillna(0) if VF_PARTY_DEM_COL in df_active.columns else 0
    df_active.loc[:, 'rep_proportion'] = (df_active[VF_PARTY_REP_COL] / df_active[VF_TOTAL_REG_COL]).fillna(0) if VF_PARTY_REP_COL in df_active.columns else 0
    
    if VF_GENERAL_2020_VOTES_COL not in df_active.columns:
        logging.error(f"Specified general election turnout column '{VF_GENERAL_2020_VOTES_COL}' not found in voter file. Cannot estimate votes.")
        return

    df_active.loc[:, 'Estimated_Dem_Votes_2020'] = (df_active[VF_GENERAL_2020_VOTES_COL] * df_active['dem_proportion']).fillna(0)
    df_active.loc[:, 'Estimated_Rep_Votes_2020'] = (df_active[VF_GENERAL_2020_VOTES_COL] * df_active['rep_proportion']).fillna(0)

    # Aggregate voter data to tracts
    logging.info("Aggregating voter data to Census Tracts...")
    tract_voter_agg_cols = {
        'total_reg_tract': (VF_TOTAL_REG_COL, 'sum'),
        'voted_general_2020_tract': (VF_GENERAL_2020_VOTES_COL, 'sum'),
        'estimated_dem_votes_tract': ('Estimated_Dem_Votes_2020', 'sum'),
        'estimated_rep_votes_tract': ('Estimated_Rep_Votes_2020', 'sum')
    }
    if VF_PARTY_DEM_COL in df_active.columns: tract_voter_agg_cols['party_dem_tract'] = (VF_PARTY_DEM_COL, 'sum')
    if VF_PARTY_REP_COL in df_active.columns: tract_voter_agg_cols['party_rep_tract'] = (VF_PARTY_REP_COL, 'sum')
    if VF_PARTY_NPP_COL in df_active.columns: tract_voter_agg_cols['party_npp_tract'] = (VF_PARTY_NPP_COL, 'sum')
    
    # Create the aggregation dictionary from the valid columns
    final_tract_voter_agg_dict = {new_col: (orig_col, agg_func) for new_col, (orig_col, agg_func) in tract_voter_agg_cols.items()}

    tract_voter_agg = df_active.groupby('census_tract').agg(**final_tract_voter_agg_dict).reset_index()
    logging.info(f"Aggregated voter data for {len(tract_voter_agg)} tracts.")

    # 2. Load CVAP Data
    logging.info(f"Loading CVAP data: {CVAP_FILE_PATH}")
    try:
        df_cvap = pd.read_csv(CVAP_FILE_PATH)
    except FileNotFoundError:
        logging.error(f"CVAP file not found at {CVAP_FILE_PATH}. Please check the path.")
        return

    df_cvap.loc[:, CVAP_BLOCK_ID_COL] = df_cvap[CVAP_BLOCK_ID_COL].astype(str).str.zfill(15)
    if CVAP_TOTAL_COL not in df_cvap.columns:
        logging.error(f"Total CVAP column '{CVAP_TOTAL_COL}' not found in CVAP file.")
        return
    df_cvap = df_cvap[df_cvap[CVAP_TOTAL_COL] > 0].copy()
    df_cvap.loc[:, 'census_tract'] = df_cvap[CVAP_BLOCK_ID_COL].str[:11]

    logging.info("Aggregating CVAP data to Census Tracts and calculating percentages...")
    cvap_agg_base = {CVAP_TOTAL_COL: 'sum', CVAP_WHITE_COL: 'sum'} # Base always needed
    for group_key in MINORITY_GROUPS_FOR_ANALYSIS.keys():
        cvap_col_for_group = CVAP_COLUMN_MAP_FOR_GROUPS.get(group_key)
        if cvap_col_for_group and cvap_col_for_group in df_cvap.columns:
            cvap_agg_base[cvap_col_for_group] = 'sum'
        else:
            logging.warning(f"CVAP column for group '{group_key}' (expected: {cvap_col_for_group}) not found in CVAP file. It will be missing from tract CVAP data.")

    tract_cvap = df_cvap.groupby('census_tract').agg(
        {k: v for k,v in cvap_agg_base.items() if k in df_cvap.columns} # Only aggregate existing columns
    ).reset_index()

    for group_key, pct_col_name_target in MINORITY_GROUPS_FOR_ANALYSIS.items():
        raw_cvap_col_for_group = CVAP_COLUMN_MAP_FOR_GROUPS.get(group_key)
        if raw_cvap_col_for_group and raw_cvap_col_for_group in tract_cvap.columns and \
           CVAP_TOTAL_COL in tract_cvap.columns and tract_cvap[CVAP_TOTAL_COL].nunique() > 0 : # Check for non-zero total
            tract_cvap[pct_col_name_target] = (tract_cvap[raw_cvap_col_for_group] / tract_cvap[CVAP_TOTAL_COL].replace(0, np.nan) * 100).fillna(0)
        else:
            logging.warning(f"Could not calculate {pct_col_name_target}. Raw CVAP column '{raw_cvap_col_for_group}' or total '{CVAP_TOTAL_COL}' missing or all zeros in tract_cvap.")
            tract_cvap[pct_col_name_target] = 0.0
    logging.info(f"Aggregated CVAP data for {len(tract_cvap)} tracts.")

    # 3. Load Shapefiles
    logging.info(f"Loading Tract shapefile: {TRACT_SHAPEFILE_PATH}")
    gdf_tract_shapes = None
    try:
        gdf_tract_shapes = gpd.read_file(TRACT_SHAPEFILE_PATH)
    except Exception as e:
        logging.error(f"Could not load Tract shapefile: {e}. Ensure path is correct and all shapefile components (.shp, .dbf, .shx, etc.) are present.")
        return
    gdf_tract_shapes.loc[:, 'census_tract'] = gdf_tract_shapes[TRACT_SHP_ID_COL].astype(str).str.zfill(11)

    logging.info(f"Loading District shapefile: {DISTRICT_SHAPEFILE_PATH}")
    gdf_districts = None
    try:
        gdf_districts = gpd.read_file(DISTRICT_SHAPEFILE_PATH)
    except Exception as e:
        logging.error(f"Could not load District shapefile: {e}. Ensure path is correct and all components are present.")
        return
    
    if DISTRICT_SHP_ID_COL not in gdf_districts.columns:
        logging.error(f"District ID column '{DISTRICT_SHP_ID_COL}' not found in district shapefile. Available columns: {gdf_districts.columns.tolist()}")
        # Attempt to find a common alternative if 'DISTRICT' is specified but not found
        if DISTRICT_SHP_ID_COL == 'DISTRICT':
            common_alternatives = ['DISTRICTNO', 'DIST_NUM', 'CD118FP', 'GEOID'] # Add more as needed
            for alt_col in common_alternatives:
                if alt_col in gdf_districts.columns:
                    logging.warning(f"'{DISTRICT_SHP_ID_COL}' not found, using alternative '{alt_col}' as district ID.")
                    DISTRICT_SHP_ID_COL = alt_col # Update global for the rest of the script
                    break
            else: # If no alternative found
                 logging.error("No suitable alternative district ID column found. Please check shapefile attributes.")
                 return
        else: # If a specific non-DISTRICT column was given and not found
            return

    gdf_districts.loc[:, DISTRICT_SHP_ID_COL] = gdf_districts[DISTRICT_SHP_ID_COL].astype(str).str.replace(r'\D+', '', regex=True).fillna('0').astype(int)


    # 4. Merge Data
    logging.info("Merging tract-level data...")
    gdf_tracts_data = pd.merge(tract_voter_agg, tract_cvap, on='census_tract', how='left')
    gdf_tracts_merged = pd.merge(gdf_tract_shapes, gdf_tracts_data, on='census_tract', how='inner')
    logging.info(f"Merged data for {len(gdf_tracts_merged)} tracts with geometries.")
    if gdf_tracts_merged.empty:
        logging.error("No tracts remained after merging all data sources. Check join keys ('census_tract') and data content.")
        return

    # 5. Spatial Join
    logging.info("Performing spatial join (assigning tracts to districts)...")
    if gdf_districts.crs != gdf_tracts_merged.crs:
        logging.info(f"Reprojecting tracts from {gdf_tracts_merged.crs} to {gdf_districts.crs} for spatial join.")
        gdf_tracts_merged = gdf_tracts_merged.to_crs(gdf_districts.crs)
    
    gdf_tracts_with_district = gpd.sjoin(gdf_tracts_merged, gdf_districts[[DISTRICT_SHP_ID_COL, 'geometry']],
                                         how='left', predicate='intersects') # Using default lsuffix, rsuffix
    # Clean up suffixes if they occur (e.g. index_right) before dropna
    if 'index_right' in gdf_tracts_with_district.columns: # Common suffix from sjoin
        gdf_tracts_with_district.rename(columns={'index_right': 'sjoin_index'}, inplace=True)

    gdf_tracts_with_district.dropna(subset=[DISTRICT_SHP_ID_COL], inplace=True)
    gdf_tracts_with_district = gdf_tracts_with_district.drop_duplicates(subset=['census_tract'], keep='first')
    gdf_tracts_with_district.loc[:, DISTRICT_SHP_ID_COL] = gdf_tracts_with_district[DISTRICT_SHP_ID_COL].astype(int)
    logging.info(f"Assigned {len(gdf_tracts_with_district)} tracts to districts.")
    if gdf_tracts_with_district.empty:
        logging.error("No tracts were successfully assigned to districts. Check CRS and spatial join predicate.")
        return

    # 6. Aggregate Data to District Level
    logging.info("Aggregating data to district level...")
    # Define base aggregation dictionary
    agg_to_dist_base = {
        'total_reg_tract': 'sum', 
        'voted_general_2020_tract': 'sum', 
        'estimated_dem_votes_tract': 'sum',
        'estimated_rep_votes_tract': 'sum',
        CVAP_TOTAL_COL: 'sum' # Raw total CVAP count
    }
    # Add optional party registration sums
    if 'party_dem_tract' in gdf_tracts_with_district.columns: agg_to_dist_base['party_dem_tract'] = 'sum'
    if 'party_rep_tract' in gdf_tracts_with_district.columns: agg_to_dist_base['party_rep_tract'] = 'sum'
    if 'party_npp_tract' in gdf_tracts_with_district.columns: agg_to_dist_base['party_npp_tract'] = 'sum'

    # Add raw CVAP counts for specified minority groups for district aggregation
    for group_key in MINORITY_GROUPS_FOR_ANALYSIS.keys():
        raw_cvap_col_for_group = CVAP_COLUMN_MAP_FOR_GROUPS.get(group_key)
        if raw_cvap_col_for_group and raw_cvap_col_for_group in gdf_tracts_with_district.columns:
            agg_to_dist_base[raw_cvap_col_for_group] = 'sum'

    # Filter for columns that actually exist in the dataframe to avoid KeyErrors
    final_agg_to_dist_dict = {k: v for k, v in agg_to_dist_base.items() if k in gdf_tracts_with_district.columns}
    if not final_agg_to_dist_dict:
        logging.error("No columns available for district aggregation. Check previous steps.")
        return

    district_aggregates = gdf_tracts_with_district.groupby(DISTRICT_SHP_ID_COL).agg(final_agg_to_dist_dict).reset_index()
    gdf_districts_final = pd.merge(gdf_districts, district_aggregates, on=DISTRICT_SHP_ID_COL, how='left')
    gdf_districts_final.fillna(0, inplace=True)

    # Calculate overall district CVAP percentages for context and export
    for group_key in MINORITY_GROUPS_FOR_ANALYSIS.keys():
        dist_pct_col_name = f'pct_cvap_{group_key}_dist'
        raw_cvap_col_for_group = CVAP_COLUMN_MAP_FOR_GROUPS.get(group_key)
        if raw_cvap_col_for_group and raw_cvap_col_for_group in gdf_districts_final.columns and \
           CVAP_TOTAL_COL in gdf_districts_final.columns and gdf_districts_final[CVAP_TOTAL_COL].sum() > 0:
            gdf_districts_final[dist_pct_col_name] = (gdf_districts_final[raw_cvap_col_for_group] / gdf_districts_final[CVAP_TOTAL_COL].replace(0, np.nan) * 100).fillna(0)
        else:
            gdf_districts_final[dist_pct_col_name] = 0.0
            logging.warning(f"Could not calculate overall district {group_key} CVAP %. Check columns: {raw_cvap_col_for_group}, {CVAP_TOTAL_COL} in gdf_districts_final")

    # 7. Calculate Gerrymandering Metrics
    pp_scores_df = calculate_polsby_popper(gdf_districts_final, DISTRICT_SHP_ID_COL, TARGET_CRS_EPSG)
    gdf_districts_final = pd.merge(gdf_districts_final, pp_scores_df, on=DISTRICT_SHP_ID_COL, how='left')

    efficiency_gap_value, eg_details_df = calculate_efficiency_gap(
        gdf_districts_final, DISTRICT_SHP_ID_COL,
        'estimated_dem_votes_tract', 'estimated_rep_votes_tract' # These are the aggregated sums
    )

    # Prepare pct columns for community concentration function
    # These are the 'pct_black', 'pct_hispanic' etc. columns on the *tract* level data
    minority_pct_cols_for_func = {key: val for key, val in MINORITY_GROUPS_FOR_ANALYSIS.items() if val in gdf_tracts_with_district.columns}
    if not minority_pct_cols_for_func:
        logging.warning("No valid CVAP percentage columns found in tract data for community concentration. Skipping.")
        comm_conc_summary_df = pd.DataFrame()
        comm_conc_details_df = pd.DataFrame()
    else:
        comm_conc_summary_df, comm_conc_details_df = calculate_community_concentration(
            gdf_tracts_with_district, DISTRICT_SHP_ID_COL,
            minority_pct_cols_for_func,
            COMMUNITY_THRESHOLD_PCT
        )

    # 8. Output Results
    logging.info("\n--- METRIC SUMMARY ---")
    logging.info(f"State: {STATE_NAME}")

    if 'polsby_popper' in gdf_districts_final.columns:
        logging.info("\nPolsby-Popper Scores:\n" + gdf_districts_final[[DISTRICT_SHP_ID_COL, 'polsby_popper']].round(4).to_string(index=False))
    else:
        logging.warning("Polsby-Popper scores not available in final district data.")

    logging.info(f"\nEfficiency Gap: {efficiency_gap_value:.4f}")
    if efficiency_gap_value < -0.0001: logging.info(f"Interpretation: Favors Republicans by {abs(efficiency_gap_value):.2%}")
    elif efficiency_gap_value > 0.0001: logging.info(f"Interpretation: Favors Democrats by {efficiency_gap_value:.2%}")
    else: logging.info("Interpretation: Neutral")

    logging.info("\nCommunity Concentration Summary:")
    if not comm_conc_summary_df.empty:
        summary_display = comm_conc_summary_df.copy()
        if 'Concentration Score (Sum Sq Shares)' in summary_display.columns:
            summary_display['Concentration Score (Sum Sq Shares)'] = summary_display['Concentration Score (Sum Sq Shares)'].map('{:.3f}'.format)
        if 'Baseline Even Spread' in summary_display.columns:
            summary_display['Baseline Even Spread'] = summary_display['Baseline Even Spread'].map('{:.3f}'.format)
        logging.info("\n" + summary_display[['Minority Group', 'Total High-Conc Tracts Statewide', 'Concentration Score (Sum Sq Shares)', 'Baseline Even Spread', 'Interpretation']].to_string(index=False))
    else:
        logging.info("Community concentration analysis was skipped or produced no results.")

    # Save detailed tables
    if not eg_details_df.empty:
        eg_details_df.to_csv(os.path.join(RESULTS_DIR, f"{STATE_ABBR}_efficiency_gap_details.csv"), index=False)
    
    if not comm_conc_details_df.empty:
        # Merge comm_conc_details_df with overall district CVAP % for the contextual view
        context_cols_to_select = [DISTRICT_SHP_ID_COL] + \
                                 [f'pct_cvap_{gk}_dist' for gk in MINORITY_GROUPS_FOR_ANALYSIS.keys() if f'pct_cvap_{gk}_dist' in gdf_districts_final.columns]
        
        # Ensure comm_conc_details_df has the district ID column with the correct name for merging
        if DISTRICT_SHP_ID_COL not in comm_conc_details_df.columns and 'DISTRICT' in comm_conc_details_df.columns:
            comm_conc_details_df.rename(columns={'DISTRICT': DISTRICT_SHP_ID_COL}, inplace=True)
        
        if DISTRICT_SHP_ID_COL in comm_conc_details_df.columns and DISTRICT_SHP_ID_COL in gdf_districts_final.columns:
            contextual_comm_conc_df = pd.merge(
                gdf_districts_final[context_cols_to_select],
                comm_conc_details_df,
                on=DISTRICT_SHP_ID_COL,
                how='left'
            )
            contextual_comm_conc_df.to_csv(os.path.join(RESULTS_DIR, f"{STATE_ABBR}_community_concentration_details_contextual.csv"), index=False)
        else:
            logging.warning(f"Could not create contextual community concentration CSV due to missing district ID column ('{DISTRICT_SHP_ID_COL}') in one of the DataFrames.")


    # --- Create State Summary JSON for Frontend ---
    state_summary_data = {
        "state_abbr": STATE_ABBR,
        "state_name": STATE_NAME,
        "efficiency_gap": round(efficiency_gap_value, 4) if pd.notna(efficiency_gap_value) else None,
        "efficiency_gap_interpretation": (
            f"Favors Republicans by {abs(efficiency_gap_value):.2%}" if efficiency_gap_value < -0.0001 else
            f"Favors Democrats by {efficiency_gap_value:.2%}" if efficiency_gap_value > 0.0001 else
            "Neutral"
        ),
        "community_concentration": []
    }
    if not comm_conc_summary_df.empty:
        for _, row in comm_conc_summary_df.iterrows():
            state_summary_data["community_concentration"].append({
                "group": row["Minority Group"],
                "score": round(row["Concentration Score (Sum Sq Shares)"], 3) if pd.notna(row["Concentration Score (Sum Sq Shares)"]) else None,
                "baseline": round(row["Baseline Even Spread"], 3) if pd.notna(row["Baseline Even Spread"]) else None,
                "interpretation": row["Interpretation"],
                "total_high_conc_tracts": row["Total High-Conc Tracts Statewide"]
            })
    
    with open(os.path.join(RESULTS_DIR, f"{STATE_ABBR}_state_summary.json"), 'w') as f:
        json.dump(state_summary_data, f, indent=4)
    logging.info(f"Saved state summary JSON to {RESULTS_DIR}")


    # Save final GeoDataFrame with scores for potential front-end use
    cols_to_export = [DISTRICT_SHP_ID_COL, 'geometry']
    if 'polsby_popper' in gdf_districts_final.columns: cols_to_export.append('polsby_popper')
    if 'estimated_dem_votes_tract' in gdf_districts_final.columns: cols_to_export.append('estimated_dem_votes_tract')
    if 'estimated_rep_votes_tract' in gdf_districts_final.columns: cols_to_export.append('estimated_rep_votes_tract')
    if CVAP_TOTAL_COL in gdf_districts_final.columns: cols_to_export.append(CVAP_TOTAL_COL)

    for group_key in MINORITY_GROUPS_FOR_ANALYSIS.keys():
        dist_pct_col = f'pct_cvap_{group_key}_dist'
        if dist_pct_col in gdf_districts_final.columns:
            cols_to_export.append(dist_pct_col)
    
    final_export_cols = [col for col in cols_to_export if col in gdf_districts_final.columns]
    if 'geometry' in gdf_districts_final.columns and DISTRICT_SHP_ID_COL in gdf_districts_final.columns:
        gdf_districts_final[final_export_cols].to_file(os.path.join(RESULTS_DIR, f"{STATE_ABBR}_districts_metrics.geojson"), driver="GeoJSON")
        logging.info(f"Saved detailed district metrics to GeoJSON in {RESULTS_DIR}")
    else:
        logging.warning("Could not save GeoJSON: 'geometry' or district ID column missing in final district data.")

    # Plot compactness scores
    if 'polsby_popper' in gdf_districts_final.columns and 'geometry' in gdf_districts_final.columns:
        try:
            fig, ax = plt.subplots(1, 1, figsize=(12, 10))
            gdf_districts_final.plot(column='polsby_popper', ax=ax, legend=True,
                                     legend_kwds={'label': "Polsby-Popper Score", 'orientation': "horizontal"},
                                     missing_kwds={"color": "lightgrey", "label": "Missing data"})
            ax.set_title(f"{STATE_NAME} Congressional District Compactness (Polsby-Popper)\nTarget CRS for calc: {TARGET_CRS_EPSG}")
            plt.savefig(os.path.join(RESULTS_DIR, f"{STATE_ABBR}_compactness_map.png"))
            logging.info(f"Saved compactness map to {RESULTS_DIR}")
            plt.close(fig) # Close the figure to free memory
        except Exception as e:
            logging.error(f"Could not plot or save compactness map: {e}")
    else:
        logging.warning("Could not plot compactness map: 'polsby_popper' or 'geometry' column missing.")

    logging.info(f"Analysis for {STATE_NAME} complete. Results in {RESULTS_DIR}")

if __name__ == "__main__":
    # This allows the script to be run from the command line.
    # For use in a Jupyter notebook, you would typically call main() directly
    # after setting the configuration variables at the top.
    # Example:
    # STATE_ABBR = "CA"; STATE_NAME = "California"; ... (set all configs) ...
    # main()
    
    # If you want to make it runnable with arguments (more advanced setup):
    # import argparse
    # parser = argparse.ArgumentParser(description=f"Run gerrymandering analysis for {STATE_NAME}.")
    # You would then parse arguments to override default config values if needed.
    # For now, direct modification of the config section is the primary method.
    
    # To run directly for the state configured at the top:
    main()


2025-05-12 14:58:00,798 - INFO - Starting analysis for Texas (TX)
2025-05-12 14:58:00,798 - INFO - Loading voter file: ../StateData/TexasData/TX_2022_prim_l2_vf_2020blocks/TX_2022_prim_l2_vf_2020blocks.csv
/var/folders/z9/gzbgl18s2wgcr125qq0q2h280000gn/T/ipykernel_581/944456676.py:283: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['480019501001000' '480019501001001' '480019501001002' ...
 '485079503025022' '485079503025023' '485079503025024']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_active.loc[:, VF_BLOCK_ID_COL] = df_active[VF_BLOCK_ID_COL].astype(str).str.zfill(15)
2025-05-12 14:58:05,797 - INFO - Aggregating voter data to Census Tracts...
2025-05-12 14:58:05,853 - INFO - Aggregated voter data for 6874 tracts.
2025-05-12 14:58:05,854 - INFO - Loading CVAP data: ../StateData/TexasData/tx_cvap_2022_2020_b/tx_cvap_2022_2020_b.csv
/var/folders/z9/gzbgl18s2wgcr125qq